In [1]:
import cv2 as cv 
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mp_detect_landmarks(image, model):
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    return image, results

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def grab_keypoints(results):
    pose = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(33*4)
    lhand = np.array([[result.x, result.y, result.z] for result in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 
    rhand = np.array([[result.x, result.y, result.z] for result in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3) 
    return np.concatenate([pose,lhand,rhand])

In [6]:
main_dir = 'C:\\Users\\nguye\\PycharmProjects\\DPL302\\Sign language\\actions'
actions = np.array(['hello','thank you','sorry','goodbye'])
sequence_size = 25
sequence_length = 30

In [8]:
for action in actions:
    for sequence in range(sequence_size):
        try:
            os.makedirs(os.path.join(main_dir, action, str(sequence)))
        except:
            pass

In [62]:
video = cv.VideoCapture(0)
is_collecting = 0
action = 0
with mp_holistic.Holistic(static_image_mode= False, min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while video.isOpened():
        ret, frame = video.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = holistic.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        draw_landmarks(frame, results)
        
        cv.putText(frame, 'Idling', (15,12),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv.LINE_AA)
        
        cv.imshow('Camera feed',frame)
        
        if cv.waitKey(5) & 0xFF == ord('q'):
            break
              
        if cv.waitKey(5) & 0xFF == ord('z'):
            action = 0
            is_collecting = 1
        
        if cv.waitKey(5) & 0xFF == ord('x'):
            action = 1
            is_collecting = 1
        
        if cv.waitKey(5) & 0xFF == ord('c'):
            action = 2
            is_collecting = 1
        
        if cv.waitKey(5) & 0xFF == ord('v'):
            action = 3
            is_collecting = 1
            
            
        if is_collecting == 1:
            for sequence in range(sequence_size):
                for frame_num in range(sequence_length):
                    ret, frame = video.read()

                    frame.flags.writeable = False
                    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                    results = holistic.process(frame)
                    frame.flags.writeable = True
                    frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

                    draw_landmarks(frame, results)
                    
                    if frame_num == min(range(sequence_size)):
                        cv.putText(frame, 'GET READY', (120,200),
                                  cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv.LINE_AA)
                        cv.putText(frame, 'Collecting: {} - Sequence {}'.format(actions[action], sequence), (15,12),
                                  cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv.LINE_AA)
                        cv.imshow('Camera feed', frame)
                        cv.waitKey(2000)
                    else:
                        cv.putText(frame, 'Collecting: {} - Sequence {}'.format(actions[action], sequence), (15,12),
                                  cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv.LINE_AA)
                        cv.imshow('Camera feed', frame)
                        
                    keypoints = grab_keypoints(results)
                    kp_path = os.path.join(main_dir, actions[action], str(sequence), str(frame_num))
                    np.save(kp_path, keypoints)
                    
                    if frame_num == max(range(sequence_size)):
                        is_collecting = 0
                    
                    if cv.waitKey(5) & 0xFF == ord('q'):
                        break

                        
video.release()
cv.destroyAllWindows()

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'hello': 0, 'thank you': 1, 'sorry': 2, 'goodbye': 3}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(sequence_size):
        window = []
        for frame_num in range(sequence_length):
            result = np.load(os.path.join(main_dir, action, str(sequence), '{}.npy'.format(frame_num)))
            window.append(result)
        sequences.append(window)
        labels.append(label_map[action])

In [18]:
np.array(sequences).shape

(100, 30, 258)

In [19]:
np.array(labels).shape

(100,)

In [20]:
X = np.array(sequences)
X.shape

(100, 30, 258)

In [21]:
y = to_categorical(labels).astype(int)
y.shape

(100, 4)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [53]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation='relu', input_shape = (30, 258)))
model.add(LSTM(128, return_sequences = True, activation='relu'))
model.add(LSTM(64, return_sequences = False, activation='relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

In [54]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.fit(X_train, y_train, epochs = 80)

Epoch 1/80
3/3 [==============================] - 6s 53ms/step - loss: 1.3791 - categorical_accuracy: 0.4000
Epoch 2/80
3/3 [==============================] - 0s 51ms/step - loss: 1.2785 - categorical_accuracy: 0.4556
Epoch 3/80
3/3 [==============================] - 0s 50ms/step - loss: 1.2716 - categorical_accuracy: 0.4333
Epoch 4/80
3/3 [==============================] - 0s 54ms/step - loss: 1.0678 - categorical_accuracy: 0.4778
Epoch 5/80
3/3 [==============================] - 0s 53ms/step - loss: 1.0150 - categorical_accuracy: 0.5222
Epoch 6/80
3/3 [==============================] - 0s 51ms/step - loss: 1.0926 - categorical_accuracy: 0.8000
Epoch 7/80
3/3 [==============================] - 0s 50ms/step - loss: 0.8151 - categorical_accuracy: 0.6333
Epoch 8/80
3/3 [==============================] - 0s 55ms/step - loss: 0.9616 - categorical_accuracy: 0.5556
Epoch 9/80
3/3 [==============================] - 0s 50ms/step - loss: 0.8725 - categorical_accuracy: 0.8333
Epoch 10/80
3/3 [==

3/3 [==============================] - 0s 51ms/step - loss: 0.1556 - categorical_accuracy: 0.9444
Epoch 76/80
3/3 [==============================] - 0s 50ms/step - loss: 0.1375 - categorical_accuracy: 0.9556
Epoch 77/80
3/3 [==============================] - 0s 51ms/step - loss: 0.1168 - categorical_accuracy: 0.9556
Epoch 78/80
3/3 [==============================] - 0s 58ms/step - loss: 0.1067 - categorical_accuracy: 0.9667
Epoch 79/80
3/3 [==============================] - 0s 49ms/step - loss: 0.1314 - categorical_accuracy: 0.9667
Epoch 80/80
3/3 [==============================] - 0s 50ms/step - loss: 0.0939 - categorical_accuracy: 0.9667


In [57]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 30, 64)            82688     
                                                                 
 lstm_16 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 4)                 132       
                                                                 
Total params: 237,284
Trainable params: 237,284
Non-tr

In [72]:
video = cv.VideoCapture(0)
sequence = []
sentence = []
threshold = 0.6

with mp_holistic.Holistic(static_image_mode= False, min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while video.isOpened():
        ret, frame = video.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = holistic.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        draw_landmarks(frame, results)
        
        cv.putText(frame, 'Testing', (15,12),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv.LINE_AA)
        
        keypoints = grab_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
         
        if len(sequence) == 30:
            result = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(result)])
        
        
        if result[np.argmax(result)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(result)] != sentence[-1]:
                    sentence.append(actions[np.argmax(result)])
            else:
                sentence.append(actions[np.argmax(result)])
                    
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv.rectangle(frame, (0,0), (640,40), (245, 117, 16), -1)
        cv.putText(frame, ' '.join(sentence), (3,30),
                  cv.FONT_HERSHEY_SIMPLEX, 1, (225, 225, 225), 2, cv.LINE_AA)
        
        
        cv.imshow('Camera feed',frame)
        
        if cv.waitKey(5) & 0xFF == ord('q'):
            break

                        
video.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step
thank you
1/1 [==============================] - 0s 27ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [==============================] - 0s 30ms/step
thank you
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 27ms/step
thank you
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [==============================] - 0s 33ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [==============================] - 0s 32ms/step
thank you
1/1 [==============================] - 0s 30ms/step
thank you
1/1 [==============================] - 0s 30ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [===

1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 30ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 30ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 30ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 33ms/step
goodbye
1/1 [==============================] - 0

goodbye
1/1 [==============================] - 0s 30ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 27ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 30ms/step
goodbye
1/1 [==============================] - 0s 31ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 31ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 29ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [=====================

sorry
1/1 [==============================] - 0s 27ms/step
sorry
1/1 [==============================] - 0s 27ms/step
sorry
1/1 [==============================] - 0s 30ms/step
sorry
1/1 [==============================] - 0s 32ms/step
sorry
1/1 [==============================] - 0s 27ms/step
sorry
1/1 [==============================] - 0s 29ms/step
sorry
1/1 [==============================] - 0s 30ms/step
sorry
1/1 [==============================] - 0s 29ms/step
sorry
1/1 [==============================] - 0s 30ms/step
sorry
1/1 [==============================] - 0s 27ms/step
sorry
1/1 [==============================] - 0s 28ms/step
sorry
1/1 [==============================] - 0s 27ms/step
sorry
1/1 [==============================] - 0s 30ms/step
sorry
1/1 [==============================] - 0s 30ms/step
thank you
1/1 [==============================] - 0s 28ms/step
thank you
1/1 [==============================] - 0s 27ms/step
thank you
1/1 [==============================] - 0s 28ms/step
th

1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 30ms/step
hello
1/1 [==============================] - 0s 27ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 27ms/step
hello
1/1 [==============================] - 0s 28ms/step
goodbye
1/1 [==============================] - 0s 29ms/step
goodbye
